In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Bidirectional
from keras.optimizers import SGD, Adam, RMSprop, Adadelta, Adagrad
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import math
from sklearn.metrics import mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Marek/Uni/Seminar_25/time-series_data.csv')

In [4]:
dataset.dropna(inplace=True)

In [5]:
#Spalte 'week' in datetime konvertieren
dataset['week'] = pd.to_datetime(dataset['week'], format='%d/%m/%y')

In [6]:
# Vorbereitung der Variablen

numeric_features = ['units_sold', 'total_price', 'base_price']
categorical_features = ['store_id', 'sku_id']

# Label Encoding für kategoriale Features
le_store = LabelEncoder()
le_sku = LabelEncoder()

dataset['store_id_encoded'] = le_store.fit_transform(dataset['store_id'])
dataset['sku_id_encoded'] = le_sku.fit_transform(dataset['sku_id'])

# Neue Features für die Sequenzierung, inklusive der binären Variablen
features = numeric_features + ['is_featured_sku', 'is_display_sku', 'store_id_encoded', 'sku_id_encoded']

In [7]:
# Erstellung Trainingsset

# Schritt 1: Eindeutige 'week'-Werte extrahieren
unique_weeks = dataset.index.unique()
n_weeks = len(unique_weeks)

weeks_train = int(0.75 * n_weeks)

# Schritt 2: Definiere die Grenzen für die Sets
train_weeks = unique_weeks[:weeks_train]
rest_weeks = unique_weeks[weeks_train:]

# Schritt 3: Daten in die jeweiligen Sets aufteilen
train_set = dataset[dataset.index.isin(train_weeks)]
print("Trainingsset:")
print(f"Anzahl der Reihen: {train_set.shape[0]}, mögliche Länge der Sequenzen: {train_set.index.nunique()}")

rest_set = dataset[dataset.index.isin(rest_weeks)]
print("Rest-Set:")
print(f"Anzahl der Reihen: {rest_set.shape[0]}, mögliche Länge der Sequenzen: {rest_set.index.nunique()}")

Trainingsset:
Anzahl der Reihen: 112611, mögliche Länge der Sequenzen: 112611
Rest-Set:
Anzahl der Reihen: 37538, mögliche Länge der Sequenzen: 37538


In [8]:
# Erstellung Validierungs- und Testset

# Schritt 1: Gruppierung nach 'store_id' und 'sku_id'
rest_grouped = list(rest_set.groupby(['store_id', 'sku_id']))

# Schritt 2: Shuffle der Gruppen, um eine zufällige Verteilung zu gewährleisten
np.random.seed(42)  # für Reproduzierbarkeit
np.random.shuffle(rest_grouped)

# Schritt 3: Aufteilung der Gruppen in Sets
n_groups = len(rest_grouped)
split = int(0.5 * n_groups)

val_groups = rest_grouped[:split]
test_groups = rest_grouped[split:]

# Schritt 4: Neue DataFrames für die Sets erstellen
val_set = pd.concat([group for _, group in val_groups])
test_set = pd.concat([group for _, group in test_groups])

# Überprüfung der Größen
print("Validierungsset:")
print(f"Anzahl der Reihen: {val_set.shape[0]}, Anzahl der Gruppen: {len(val_groups)}, Länge der Reihen: {val_set.index.nunique()}")
print("Test-Set:")
print(f"Anzahl der Reihen: {test_set.shape[0]}, Anzahl der Gruppen: {len(test_groups)}, Länge der Reihen: {test_set.index.nunique()}")

Validierungsset:
Anzahl der Reihen: 18739, Anzahl der Gruppen: 577, Länge der Reihen: 18739
Test-Set:
Anzahl der Reihen: 18799, Anzahl der Gruppen: 578, Länge der Reihen: 18799


In [9]:
# Index zurücksetzen

# Schritt 1: Den Index wieder auf den datetime 'week' setzen (falls nicht mehr so)
train_set = train_set.set_index('week')
val_set = val_set.set_index('week')
test_set = test_set.set_index('week')

# Schritt 2: Nach 'week' sortieren
train_set = train_set.sort_index()
val_set = val_set.sort_index()
test_set = test_set.sort_index()

In [10]:
print("Trainingsset:")
print(f"Anzahl der Reihen: {train_set.shape[0]}, mögliche Länge der Sequenzen: {train_set.index.nunique()}")
print()
print("Validierungsset:")
print(f"Anzahl der Reihen: {val_set.shape[0]}, Anzahl der Gruppen: {len(val_groups)}, Länge der Reihen: {val_set.index.nunique()}")
print()
print("Test-Set:")
print(f"Anzahl der Reihen: {test_set.shape[0]}, Anzahl der Gruppen: {len(test_groups)}, Länge der Reihen: {test_set.index.nunique()}")

Trainingsset:
Anzahl der Reihen: 112611, mögliche Länge der Sequenzen: 98

Validierungsset:
Anzahl der Reihen: 18739, Anzahl der Gruppen: 577, Länge der Reihen: 33

Test-Set:
Anzahl der Reihen: 18799, Anzahl der Gruppen: 578, Länge der Reihen: 33


In [11]:
print("Trainingsset:")
train_set.info()
train_set.head()

Trainingsset:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 112611 entries, 2011-01-17 to 2012-11-27
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   record_ID         112611 non-null  int64  
 1   store_id          112611 non-null  int64  
 2   sku_id            112611 non-null  int64  
 3   total_price       112611 non-null  float64
 4   base_price        112611 non-null  float64
 5   is_featured_sku   112611 non-null  int64  
 6   is_display_sku    112611 non-null  int64  
 7   units_sold        112611 non-null  int64  
 8   store_id_encoded  112611 non-null  int64  
 9   sku_id_encoded    112611 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 13.5 MB


,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,store_id_encoded,sku_id_encoded
week,,,,,,,,,,
2011-01-17,1,8091,216418,99.0375,111.8625,0,0,20,3,1
2011-01-17,2,8091,216419,99.0375,99.0375,0,0,28,3,2
2011-01-17,3,8091,216425,133.9500,133.9500,0,0,19,3,3
2011-01-17,4,8091,216233,133.9500,133.9500,0,0,44,3,0
2011-01-17,5,8091,217390,141.0750,141.0750,0,0,52,3,5


In [12]:
print("Validierungsset:")
val_set.info()
val_set.head()

Validierungsset:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18739 entries, 2012-11-27 to 2013-07-09
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   record_ID         18739 non-null  int64  
 1   store_id          18739 non-null  int64  
 2   sku_id            18739 non-null  int64  
 3   total_price       18739 non-null  float64
 4   base_price        18739 non-null  float64
 5   is_featured_sku   18739 non-null  int64  
 6   is_display_sku    18739 non-null  int64  
 7   units_sold        18739 non-null  int64  
 8   store_id_encoded  18739 non-null  int64  
 9   sku_id_encoded    18739 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 2.1 MB


,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,store_id_encoded,sku_id_encoded
week,,,,,,,,,,
2012-11-27,160008,9823,673209,355.5375,355.5375,0,0,49,63,26
2012-11-27,159508,9439,219009,190.2375,190.2375,0,0,60,41,7
2012-11-27,159667,9498,216419,86.9250,86.9250,0,0,34,47,2
2012-11-27,160007,9823,547934,177.4125,177.4125,0,0,23,63,24
2012-11-27,160288,9984,245338,391.8750,469.5375,1,0,23,75,14


In [13]:
print("Testset:")
test_set.info()
test_set.head()

Testset:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18799 entries, 2012-11-27 to 2013-07-09
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   record_ID         18799 non-null  int64  
 1   store_id          18799 non-null  int64  
 2   sku_id            18799 non-null  int64  
 3   total_price       18799 non-null  float64
 4   base_price        18799 non-null  float64
 5   is_featured_sku   18799 non-null  int64  
 6   is_display_sku    18799 non-null  int64  
 7   units_sold        18799 non-null  int64  
 8   store_id_encoded  18799 non-null  int64  
 9   sku_id_encoded    18799 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 2.1 MB


,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,store_id_encoded,sku_id_encoded
week,,,,,,,,,,
2012-11-27,160238,9954,245338,391.8750,469.5375,1,0,27,73,14
2012-11-27,160002,9823,223153,236.5500,236.5500,0,0,40,63,12
2012-11-27,160039,9809,219029,327.0375,327.0375,0,0,36,61,8
2012-11-27,159614,9479,222765,175.9875,227.2875,0,0,50,44,11
2012-11-27,159616,9479,223153,235.8375,235.8375,0,0,26,44,12


In [11]:
# Skalierer nur für die numerischen Features (ohne binäre)
scalers = {feature: MinMaxScaler() for feature in numeric_features}

for feature in numeric_features:
  scalers[feature].fit(train_set[feature].values.reshape(-1, 1))

In [12]:
# Funktion zur Erstellung von Sequenzen
def create_sequences(dataset, features, numeric_features, scalers, timesteps):
    X_all = []
    y_all = []

    grouped = dataset.groupby(['store_id', 'sku_id'])

    for (store_id, sku_id), group in grouped:
        group = group.sort_index()

        # Daten extrahieren
        data = group[features].values

        # Skalieren der numerischen Features
        for feature in numeric_features:
            index = features.index(feature)
            data[:, index] = scalers[feature].transform(data[:, index].reshape(-1, 1)).flatten()

        # Kategoriale Features bleiben als Integer

        for i in range(timesteps, len(data)):
            X_seq = data[i - timesteps:i]
            X_all.append(X_seq)
            y_all.append(data[i, 0])  # units_sold ist der erste Wert in features

    X_array = np.array(X_all)
    y_array = np.array(y_all)
    return X_array, y_array

In [13]:
# Liste der Fenstergrößen
window_sizes = [30]

# Dictionary, um die erzeugten Datensätze zu speichern
train_sets = {}
val_sets = {}

for window in window_sizes:
    X_train, y_train = create_sequences(train_set, features, numeric_features, scalers, window)
    train_sets[window] = {'X': X_train, 'y': y_train}

    X_val, y_val = create_sequences(val_set, features, numeric_features, scalers, window)
    val_sets[window] = {'X': X_val, 'y': y_val}

    print(f"Fenstergröße {window}:")
    print(f"Trainings-Sequenzen: {X_train.shape}, Validierungs-Sequenzen: {X_val.shape}.")

Fenstergröße 30:
Trainings-Sequenzen: (77961, 30, 7), Validierungs-Sequenzen: (1429, 30, 7).


In [14]:
X_tr = train_sets[30]['X']
y_tr = train_sets[30]['y']
X_v = val_sets[30]['X']
y_v = val_sets[30]['y']

In [20]:
# Hyperparameter-Optionen
layer_options = [1, 2, 3]
units_options = [32, 64, 128]

# Ergebnisse speichern
results = {}

for n_layers in layer_options:
    for units in units_options:
        print(f"\nTraining: {n_layers} Schichten, {units} Units (SimpleRNN)")

        model = Sequential()

        # Erste Schicht
        model.add(GRU(units=units, input_shape=(X_tr.shape[1], X_tr.shape[2]), return_sequences=(n_layers > 1)))

        # Zusätzliche Schichten bei Bedarf
        for layer_idx in range(1, n_layers):
            return_seq = (layer_idx < n_layers - 1)
            model.add(GRU(units=units, return_sequences=return_seq))

        # Endfolger
        model.add(Dense(1, activation='linear'))

        # Kompilieren mit SGD, Lernrate 0,01
        model.compile(optimizer=SGD(learning_rate=0.01), loss='mean_squared_error')

        # Training
        history = model.fit(X_tr, y_tr, epochs=10, batch_size=64, validation_data=(X_v, y_v))

        train_losses = history.history['loss']
        val_losses = history.history['val_loss']

        # Resultate speichern
        key = (n_layers, units)
        results[key] = {'train_loss': train_losses, 'val_loss': val_losses}

        # Ausgabe
        print(f"Durchschnittlicher Validierungsverlust: {np.mean(val_losses):.6f}")
        print(f"Bester Validierungsverlust: {np.min(val_losses):.6f}")


Training: 1 Schichten, 32 Units (SimpleRNN)
Epoch 1/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - loss: 0.0107 - val_loss: 0.0012
Epoch 2/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 40s 16ms/step - loss: 9.6377e-04 - val_loss: 8.9599e-04
Epoch 3/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - loss: 7.0708e-04 - val_loss: 7.7317e-04
Epoch 4/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 6.2672e-04 - val_loss: 7.2581e-04
Epoch 5/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - loss: 5.7107e-04 - val_loss: 7.8099e-04
Epoch 6/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - loss: 5.3385e-04 - val_loss: 6.2276e-04
Epoch 7/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - loss: 4.9906e-04 - val_loss: 6.9106e-04
Epoch 8/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - loss: 4.5793e-04 - val_loss: 5.4019e-04
Epoch 9/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 4.7416e-04 - val_loss: 5.2806e-04
Epoch 10/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - loss: 4.4880e-04 - val

In [18]:
# Liste der Optimizer und Lernraten
optimizers = {
    'Adam': Adam,
    'RMSprop': RMSprop,
    'Adadelta': Adadelta,
    'Adagrad': Adagrad
}

learning_rates = [0.001, 0.005, 0.01, 0.05, 0.1]

# Ergebnisse speichern
results = []

for opt_name, opt_func in optimizers.items():
    for lr in learning_rates:
        print(f"Testen mit Optimizer: {opt_name}, Lernrate: {lr}")

        # Modell erstellen
        model = Sequential()

        #Erste Schicht
        model.add(GRU(units=32,
                       input_shape=(X_tr.shape[1], X_tr.shape[2]),
                       return_sequences=True))

        #Zweite Schicht
        model.add(GRU(units=32, return_sequences=True))

        #Dritte Schicht
        model.add(GRU(units=32))

        model.add(Dense(1, activation='linear'))

        # Optimizer Instanziieren
        optimizer = opt_func(learning_rate=lr)

        # Modell kompilieren
        model.compile(optimizer=optimizer,
                      loss='mean_squared_error')

        # Training durchführen
        history = model.fit(X_tr, y_tr, epochs=10, batch_size=64,
                            validation_data=(X_v, y_v))

        # Validation Loss speichern
        val_loss = np.min(history.history['val_loss'])
        print(f"Validierungsverlust: {val_loss:.6f}")
        results.append({
            'optimizer': opt_name,
            'learning_rate': lr,
            'val_loss': val_loss
        })

# Ergebnisse anzeigen
for res in results:
    print(f"Optimizer: {res['optimizer']}, Lernrate: {res['learning_rate']}, "
          f"Bester Validierungsverlust: {res['val_loss']:.6f}")

Testen mit Optimizer: Adam, Lernrate: 0.001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 75s 56ms/step - loss: 0.0014 - val_loss: 7.8019e-04
Epoch 2/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 80s 55ms/step - loss: 3.9588e-04 - val_loss: 3.9316e-04
Epoch 3/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 64s 53ms/step - loss: 3.2778e-04 - val_loss: 5.1008e-04
Epoch 4/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 66s 54ms/step - loss: 2.8930e-04 - val_loss: 3.3641e-04
Epoch 5/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 55ms/step - loss: 2.5350e-04 - val_loss: 3.3038e-04
Epoch 6/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 79s 53ms/step - loss: 2.3360e-04 - val_loss: 3.3215e-04
Epoch 7/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 53ms/step - loss: 2.5030e-04 - val_loss: 3.6508e-04
Epoch 8/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 85s 56ms/step - loss: 2.3787e-04 - val_loss: 3.2163e-04
Epoch 9/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 78s 53ms/step - loss: 2.2038e-04 - val_loss: 3.5839e-04
Epoch 10/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 53ms/step - loss: 2.2263e-04 - val_loss: 3.6624e-04
Validierungsverlust: 0.

In [19]:
# Modell erstellen
model_GRU_01 = Sequential()
model_GRU_01.add(GRU(units=32,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model_GRU_01.add(GRU(units=32, return_sequences=True))
model_GRU_01.add(GRU(units=32))
model_GRU_01.add(Dense(1, activation='linear'))

# Modell kompilieren
model_GRU_01.compile(optimizer=Adam(learning_rate=0.01),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)

# Training durchführen
history = model_GRU_01.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 75s 57ms/step - loss: 0.0116 - val_loss: 3.8948e-04 - learning_rate: 0.0100
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 80s 55ms/step - loss: 3.0709e-04 - val_loss: 3.3161e-04 - learning_rate: 0.0100
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 56ms/step - loss: 2.7066e-04 - val_loss: 3.3598e-04 - learning_rate: 0.0100
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 66s 54ms/step - loss: 2.4745e-04 - val_loss: 3.3214e-04 - learning_rate: 0.0100
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 67s 55ms/step - loss: 2.1248e-04 - val_loss: 3.5897e-04 - learning_rate: 1.0000e-03
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 67s 55ms/step - loss: 2.1510e-04 - val_loss: 3.2658e-04 - learning_rate: 1.0000e-03
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 81s 54ms/step - loss: 2.1487e-04 - val_loss: 3.3304e-04 - learning_rate: 1.0000e-03
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 55ms/step - loss: 1.9226e-04 - val_loss: 3.2137e-04 - learning_rate: 1.0000e-04
Epoch 9/50
1

In [15]:
# Modell erstellen
model_GRU_001 = Sequential()
model_GRU_001.add(GRU(units=32,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model_GRU_001.add(GRU(units=32, return_sequences=True))
model_GRU_001.add(GRU(units=32))
model_GRU_001.add(Dense(1, activation='linear'))

# Modell kompilieren
model_GRU_001.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)

# Training durchführen
history = model_GRU_001.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 77s 57ms/step - loss: 0.0011 - val_loss: 4.9157e-04 - learning_rate: 0.0010
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 79s 55ms/step - loss: 3.5471e-04 - val_loss: 4.3611e-04 - learning_rate: 0.0010
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - loss: 3.0861e-04 - val_loss: 3.3209e-04 - learning_rate: 0.0010
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 56ms/step - loss: 2.7283e-04 - val_loss: 3.1914e-04 - learning_rate: 0.0010
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 66s 54ms/step - loss: 2.5980e-04 - val_loss: 3.0812e-04 - learning_rate: 0.0010
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 85s 57ms/step - loss: 2.3423e-04 - val_loss: 3.0887e-04 - learning_rate: 0.0010
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 80s 55ms/step - loss: 2.1999e-04 - val_loss: 2.9773e-04 - learning_rate: 1.0000e-04
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 68s 56ms/step - loss: 2.3397e-04 - val_loss: 2.9905e-04 - learning_rate: 1.0000e-04
Epoch 9/50
1219/1219

In [16]:
# Modell erstellen
model_GRU_01_5 = Sequential()
model_GRU_01_5.add(GRU(units=32,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model_GRU_01_5.add(GRU(units=32, return_sequences=True))
model_GRU_01_5.add(GRU(units=32))
model_GRU_01_5.add(Dense(1, activation='linear'))

# Modell kompilieren
model_GRU_01_5.compile(optimizer=Adam(learning_rate=0.01),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-8)

# Training durchführen
history = model_GRU_01_5.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 77s 58ms/step - loss: 0.0249 - val_loss: 4.2077e-04 - learning_rate: 0.0100
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 78s 55ms/step - loss: 3.4261e-04 - val_loss: 4.5443e-04 - learning_rate: 0.0100
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 54ms/step - loss: 3.0939e-04 - val_loss: 4.3293e-04 - learning_rate: 0.0100
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 84s 56ms/step - loss: 2.7193e-04 - val_loss: 4.0628e-04 - learning_rate: 0.0100
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 56ms/step - loss: 2.4058e-04 - val_loss: 3.3112e-04 - learning_rate: 0.0050
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 81s 55ms/step - loss: 2.4262e-04 - val_loss: 3.2509e-04 - learning_rate: 0.0050
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - loss: 2.2930e-04 - val_loss: 3.8830e-04 - learning_rate: 0.0050
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 67s 55ms/step - loss: 2.1230e-04 - val_loss: 3.4367e-04 - learning_rate: 0.0025
Epoch 9/50
1219/1219 ━━━━━━━

In [17]:
# Modell erstellen
model_GRU_001_5 = Sequential()
model_GRU_001_5.add(GRU(units=32,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model_GRU_001_5.add(GRU(units=32, return_sequences=True))
model_GRU_001_5.add(GRU(units=32))
model_GRU_001_5.add(Dense(1, activation='linear'))

# Modell kompilieren
model_GRU_001_5.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-8)

# Training durchführen
history = model_GRU_001_5.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 73s 55ms/step - loss: 0.0023 - val_loss: 4.7728e-04 - learning_rate: 0.0010
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 65s 54ms/step - loss: 3.6790e-04 - val_loss: 4.1960e-04 - learning_rate: 0.0010
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 81s 53ms/step - loss: 3.5397e-04 - val_loss: 3.9168e-04 - learning_rate: 0.0010
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 53ms/step - loss: 3.0429e-04 - val_loss: 3.7467e-04 - learning_rate: 0.0010
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 54ms/step - loss: 2.8056e-04 - val_loss: 3.1296e-04 - learning_rate: 0.0010
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 64s 53ms/step - loss: 2.7517e-04 - val_loss: 3.3056e-04 - learning_rate: 0.0010
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 67s 55ms/step - loss: 2.5789e-04 - val_loss: 3.1758e-04 - learning_rate: 0.0010
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 81s 54ms/step - loss: 2.3314e-04 - val_loss: 3.0535e-04 - learning_rate: 5.0000e-04
Epoch 9/50
1219/1219 ━━━